# Day 21: 動き検出

## Learning Objectives
- 動き検出の原理を理解する
- フレーム間差分法を実装する
- 光流法を理解する
- 背景差分法を実装する

---

# Part 1: Theory (2 hours)

## 1.1 動き検出とは？

動き検出は、動画シーケンスから移動しているオブジェクトや変化を検出する技術です。コンピュータビジョンの基本的なタスクの一つであり、多くの応用で使用されます。

**主な用途**:
- セキュリティ監視
- 交通監視
- ビデオ会議の背景ぼかし
- 動きベースのトリガー
- 動体検知カメラ

**基本的な考え方**:
1. 現在のフレームと過去のフレームを比較
2. 画素値の変化を検出
3. 閾値処理で動き領域を抽出
4. ノイズ除去とオブジェクト検出

## 1.2 フレーム間差分法 (Frame Differencing)

最も基本的な動き検出手法です。連続するフレーム間の差を計算して動きを検出します。

**アルゴリズム**:
1. 現在フレーム `I(t)` と前フレーム `I(t-1)` を読み込む
2. 絶対差を計算: `D(x,y) = |I(t)(x,y) - I(t-1)(x,y)|`
3. しきい値処理: `M(x,y) = 1 if D(x,y) > T else 0`
4. ノイズ除去（オープニング・クロージング）

**特徴**:
- メリット: 簡単で高速、背景の変化には頑健
- デメリット: 動きの方向情報なし、照明変化に敏感

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [ ]:
def frame_difference(frame1, frame2, threshold=30):
    """フレーム間差分を計算"""
    # 絶対差を計算
    diff = cv2.absdiff(frame1, frame2)
    
    # グレースケール化（もしされていなければ）
    if len(diff.shape) > 2:
        diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    # 2値化
    _, binary = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    
    # ノイズ除去（モルフォロジー演算）
    kernel = np.ones((3,3), np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel)
    
    return cleaned

def create_test_sequence(height=100, width=100, num_frames=30):
    """テスト用の動画シーケンスを作成"""
    frames = []
    
    # 静的な背景
    background = np.random.randint(50, 150, (height, width), dtype=np.uint8)
    
    for i in range(num_frames):
        frame = background.copy()
        
        # 移動する物体（円形）
        center_x = 30 + i * 2  # 右に移動
        center_y = 50 + 20 * np.sin(i * 0.3)  # 上下に正弦波運動
        
        # 円を描画
        y_coords, x_coords = np.ogrid[:height, :width]
        mask = (x_coords - center_x)**2 + (y_coords - center_y)**2 <= 20**2
        frame[mask] = 255  # 白い物体
        
        frames.append(frame)
    
    return frames

# テストシーケンスの作成
frames = create_test_sequence()
print(f"{len(frames)}フレームのシーケンスを作成")
print(f"画像サイズ: {frames[0].shape}")

# 最初のフレームを表示
plt.figure(figsize=(12, 8))

plt.subplot(2, 3, 1)
plt.imshow(frames[0], cmap='gray')
plt.title("フレーム 1")
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(frames[15], cmap='gray')
plt.title("フレーム 15")
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(frames[29], cmap='gray')
plt.title("フレーム 30")
plt.axis('off')

# フレーム間差分
diff1 = frame_difference(frames[0], frames[1])
diff15 = frame_difference(frames[14], frames[15])
diff29 = frame_difference(frames[28], frames[29])

plt.subplot(2, 3, 4)
plt.imshow(diff1, cmap='gray')
plt.title("フレーム差分 (1-2)")
plt.axis('off')

plt.subplot(2, 3, 5)
plt.imshow(diff15, cmap='gray')
plt.title("フレーム差分 (15-16)")
plt.axis('off')

plt.subplot(2, 3, 6)
plt.imshow(diff29, cmap='gray')
plt.title("フレーム差分 (29-30)")
plt.axis('off')

plt.tight_layout()
plt.show()

## 1.3 背景差分法 (Background Subtraction)

背景モデルを維持し、現在のフレームから背景を差し引いて前景（動き）を検出します。

**アルゴリズム**:
1. 背景モデル `B(x,y)` を作成
2. フレーム `I(t)` から背景を差し引く: `D(x,y) = |I(t)(x,y) - B(x,y)|`
3. しきい値処理で前景を抽出
4. 背景モデルの更新（オプション）

**背景モデルの更新方法**:
- 単純平均: `B_new = α × B_old + (1-α) × I(t)`
- 中値フィルタ
- ガウシアン混合モデル (GMM)

**特徴**:
- メリット: 静止したオブジェクトを除去可能、照明変化に頑健
- デメリット: 背景の変化に弱い、計算コストが高い

In [ ]:
class SimpleBackgroundSubtractor:
    """単純な背景差分器"""
    def __init__(self, alpha=0.95, threshold=30):
        self.background = None
        self.alpha = alpha  # 背景更新率（大きいほど遅く更新）
        self.threshold = threshold
    
    def update_background(self, frame):
        """背景モデルを更新"""
        if self.background is None:
            self.background = frame.copy()
        else:
            # 移動平均で背景を更新
            self.background = self.alpha * self.background + (1 - self.alpha) * frame
    
    def subtract(self, frame):
        """背景差分を計算"""
        # 絶対差を計算
        diff = cv2.absdiff(frame, self.background.astype(np.uint8))
        
        # 2値化
        _, binary = cv2.threshold(diff, self.threshold, 255, cv2.THRESH_BINARY)
        
        # ノイズ除去
        kernel = np.ones((3,3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        
        return cleaned

# 背景差分のテスト
bg_subtractor = SimpleBackgroundSubtractor(alpha=0.95)

backgrounds = []
foregrounds = []

for i in range(30):
    if i == 0:
        # 最初のフレームで背景を初期化
        bg_subtractor.update_background(frames[i])
    else:
        # 背景を更新
        bg_subtractor.update_background(frames[i])
        # フォアグラウンドを取得
        fg = bg_subtractor.subtract(frames[i])
        foregrounds.append(fg)

print(f"背景モデルの平均値: {np.mean(bg_subtractor.background):.2f}")

# 結果の可視化
plt.figure(figsize=(15, 10))

# 初期背景
plt.subplot(3, 3, 1)
plt.imshow(bg_subtractor.background, cmap='gray')
plt.title("初期背景モデル")
plt.axis('off')

# 結果を表示
for idx, frame_num in enumerate([5, 10, 15, 20, 25, 29]):
    plt.subplot(3, 3, idx + 2)
    plt.imshow(foregrounds[frame_num-1], cmap='gray')
    plt.title(f"フレーム {frame_num} の前景")
    plt.axis('off')

plt.tight_layout()
plt.show()

## 1.4 光流法 (Optical Flow)

各ピクセルの動きベクトルを計算する手法です。動きの方向と速度を検出できます。

**Lucas-Kradek法**（最も一般的）:
1. 隣接ピクセルの輝度が時間とともに一定であると仮定
2. 動きベクトル `(u,v)` を計算する方程式を導出
3. 最小二乗法でベクトルを求める

**光流方程式**:

```
I(x+u, y+v, t+1) ≈ I(x,y,t)

テイラー展開して整理:

I_x × u + I_y × v + I_t = 0

ここで：
- I_x, I_y: 空間勾配
- I_t: 時間的変化
- u, v: 動きベクトル
```

In [ ]:
def calculate_optical_flow(frame1, frame2, window_size=3):
    """簡易的な光流計算（Lucas-Kradek法の概念）"""
    # グレースケール化
    if len(frame1.shape) > 2:
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # 画像サイズ
    height, width = frame1.shape
    
    # 勾配を計算
    I_x = cv2.Sobel(frame1, cv2.CV_64F, 1, 0, ksize=3)
    I_y = cv2.Sobel(frame1, cv2.CV_64F, 0, 1, ksize=3)
    I_t = frame2.astype(np.float64) - frame1.astype(np.float64)
    
    # 光流ベクトルを計算
    flow = np.zeros((height, width, 2), dtype=np.float32)
    
    # ウィンドウサイズ内で計算
    pad = window_size // 2
    
    for i in range(pad, height - pad):
        for j in range(pad, width - pad):
            # ウィンド領域を抽出
            win_Ix = I_x[i-pad:i+pad+1, j-pad:j+pad+1].flatten()
            win_Iy = I_y[i-pad:i+pad+1, j-pad:j+pad+1].flatten()
            win_It = I_t[i-pad:i+pad+1, j-pad:j+pad+1].flatten()
            
            # 行列Aとベクトルbを作成
            A = np.column_stack([win_Ix, win_Iy])
            b = -win_It
            
            # 最小二乗解を求める
            if np.linalg.matrix_rank(A) == 2:
                u, v = np.linalg.lstsq(A, b, rcond=None)[0]
                flow[i, j] = [u, v]
            else:
                flow[i, j] = [0, 0]
    
    return flow

def visualize_flow(flow, step=10):
    """光流を可視化"""
    height, width = flow.shape[:2]
    
    # 動きベクトルをサンプリング
    h, w = np.mgrid[0:height:step, 0:width:step]
    u = flow[::step, ::step, 0]
    v = flow[::step, ::step, 1]
    
    # ベクトルの強度（速度）
    magnitude = np.sqrt(u**2 + v**2)
    
    # 色相マップで可視化
    hsv = np.zeros((height, width, 3), dtype=np.uint8)
    hsv[..., 1] = 255
    hsv[..., 0] = (magnitude / (magnitude.max() + 1e-6) * 180).astype(np.uint8)
    hsv[..., 2] = 255
    
    # HSVからBGRに変換
    flow_color = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    # ベクトルを描画
    flow_img = flow_color.copy()
    
    for i in range(0, u.shape[0]):
        for j in range(0, u.shape[1]):
            if magnitude[i, j] > 1:  # しきい値以上の動きのみ表示
                start = (j * step, i * step)
                end = (int(j * step + u[i, j]), int(i * step + v[i, j]))
                cv2.arrowLine(flow_img, start, end, (0, 255, 0), 2)
    
    return flow_img

# 光流の計算
flow = calculate_optical_flow(frames[15], frames[16])

print(f"光流ベクトルの形状: {flow.shape}")
print(f"動きの最大速度: {np.sqrt(flow[:,:,0]**2 + flow[:,:,1]**2).max():.2f}")

# 可視化
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(frames[15], cmap='gray')
plt.title("フレーム 15")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(frames[16], cmap='gray')
plt.title("フレーム 16")
plt.axis('off')

plt.subplot(1, 3, 3)
flow_color = visualize_flow(flow, step=8)
plt.imshow(flow_color)
plt.title("光流（緑色の矢印）")
plt.axis('off')

plt.tight_layout()
plt.show()

---

# Part 2: Practice (2 hours)

それでは、学んだ知識を実際に使ってみましょう！

## Exercise 21.1: フレーム間差分法の実装

フレーム間差分法を自前で実装し、動きを検出してください。

In [ ]:
def my_frame_diff(frame1, frame2, threshold=30, morphology=True):
    """自作のフレーム間差分"""
    # グレースケール化
    if len(frame1.shape) > 2:
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # 1. 絶対差を計算
    diff = np.abs(frame1.astype(float) - frame2.astype(float))
    
    # 2. 2値化
    binary = (diff > threshold).astype(np.uint8) * 255
    
    # 3. モルフォロジー処理（オプション）
    if morphology:
        kernel = np.ones((3,3), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    return binary

# テスト
my_diff1 = my_frame_diff(frames[10], frames[11], threshold=20)
cv_diff1 = frame_difference(frames[10], frames[11], threshold=20)

# 結果の比較
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(my_diff1, cmap='gray')
plt.title("自作フレーム差分")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cv_diff1, cmap='gray')
plt.title("OpenCVフレーム差分")
plt.axis('off')

plt.subplot(1, 3, 3)
diff_map = my_diff1 != cv_diff1
plt.imshow(diff_map, cmap='Reds')
plt.title("違い（赤）")
plt.axis('off')

plt.tight_layout()
plt.show()

# 一致度の確認
diff_pixels = np.sum(diff_map)
total_pixels = frames[0].shape[0] * frames[0].shape[1]
accuracy = (1 - diff_pixels / total_pixels) * 100
print(f"一致度: {accuracy:.2f}%")

## Exercise 21.2: 背景モデルの更新戦略

異なる更新戦略で背景モデルを比較してください。

In [ ]:
class MedianBackgroundSubtractor:
    """中値フィルタによる背景差分器"""
    def __init__(self, history=5, threshold=30):
        self.history_frames = []
        self.history_size = history
        self.threshold = threshold
    
    def update(self, frame):
        """フレーム履歴を更新"""
        self.history_frames.append(frame)
        if len(self.history_frames) > self.history_size:
            self.history_frames.pop(0)
    
    def get_background(self):
        """中値を背景として取得"""
        if len(self.history_frames) == 0:
            return None
        
        # スタックして中値を計算
        stack = np.stack(self.history_frames, axis=2)
        background = np.median(stack, axis=2).astype(np.uint8)
        return background
    
    def subtract(self, frame):
        """背景差分"""
        if len(self.history_frames) == 0:
            return np.zeros_like(frame)
        
        background = self.get_background()
        diff = cv2.absdiff(frame, background)
        _, binary = cv2.threshold(diff, self.threshold, 255, cv2.THRESH_BINARY)
        
        # モルフォロジー処理
        kernel = np.ones((3,3), np.uint8)
        cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        
        return cleaned

# 中値フィルタ背景差分器のテスト
median_bg_sub = MedianBackgroundSubtractor(history=5)
median_foregrounds = []

for i in range(30):
    median_bg_sub.update(frames[i])
    if i >= 4:  # 5フレーム目から開始
        fg = median_bg_sub.subtract(frames[i])
        median_foregrounds.append(fg)

# 結果の比較
plt.figure(figsize=(15, 10))

# シンプルな平均と中値の比較
simple_fg = bg_subtractor.subtract(frames[29])

plt.subplot(2, 3, 1)
plt.imshow(frames[0], cmap='gray')
plt.title("フレーム 1")
plt.axis('off')

plt.subplot(2, 3, 2)
plt.imshow(simple_fg, cmap='gray')
plt.title("平均背景差分")
plt.axis('off')

plt.subplot(2, 3, 3)
plt.imshow(median_foregrounds[-1], cmap='gray')
plt.title("中値背景差分")
plt.axis('off')

# 中間フレームの比較
plt.subplot(2, 3, 4)
plt.imshow(frames[15], cmap='gray')
plt.title("フレーム 15")
plt.axis('off')

plt.subplot(2, 3, 5)
current_simple = bg_subtractor.subtract(frames[15])
plt.imshow(current_simple, cmap='gray')
plt.title("平均背景差分 (15)")
plt.axis('off')

plt.subplot(2, 3, 6)
median_fg = median_foregrounds[15-4] if 15-4 < len(median_foregrounds) else median_foregrounds[-1]
plt.imshow(median_fg, cmap='gray')
plt.title("中値背景差分 (15)")
plt.axis('off')

plt.tight_layout()
plt.show()

## Exercise 21.3: 光流法の実践

光流法を使って物体の軌跡を追跡してください。

In [ ]:
def track_object_with_optical_flow(frames, start_pos, max_frames=10):
    """光流法による物体追跡"""
    # 追跡結果のリスト
    positions = [start_pos]
    
    # 前フレームを取得
    prev_frame = frames[0]
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    # 追跡開始位置の近傍に特徴点を選択
    x, y = start_pos
    neighborhood_size = 20
    
    for i in range(1, min(len(frames), max_frames)):
        # 現在のフレーム
        curr_frame = frames[i]
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        
        # 特徴点を検出（追跡開始位置の近傍）
        y_min = max(0, y - neighborhood_size)
        y_max = min(curr_gray.shape[0], y + neighborhood_size)
        x_min = max(0, x - neighborhood_size)
        x_max = min(curr_gray.shape[1], x + neighborhood_size)
        
        # 領域内で角点を検出
        region = curr_gray[y_min:y_max, x_min:x_max]
        corners = cv2.goodFeaturesToTrack(
            region, 1, 0.01, 10
        )
        
        if corners is not None:
            # 角点を画像座標に変換
            corner = corners[0].ravel()
            new_x = int(x_min + corner[0])
            new_y = int(y_min + corner[1])
            
            # 位置を更新
            x, y = new_x, new_y
            positions.append((x, y))
        
        # 現在のフレームを次のイテレーション用に保存
        prev_gray = curr_gray
    
    return positions

# 物体追跡のテスト
# 物体の初期位置（フレーム10での円の中心）
start_x, start_y = 50, 50  # おおよその初期位置
positions = track_object_with_optical_flow(frames, (start_x, start_y), max_frames=20)

print(f"追跡した位置: {positions}")

# 軌跡の可視化
plt.figure(figsize=(15, 10))

# 軌跡をフレーム上に描画
for idx, frame_num in enumerate([0, 10, 20]):
    plt.subplot(2, 3, idx + 1)
    plt.imshow(frames[frame_num], cmap='gray')
    
    # 位置を描画
    pos_idx = min(frame_num, len(positions) - 1)
    if pos_idx < len(positions):
        x, y = positions[pos_idx]
        plt.plot(x, y, 'ro', markersize=10)
        plt.plot(positions[:pos_idx+1], 'g-', alpha=0.5)
    
    plt.title(f"フレーム {frame_num}")
    plt.axis('off')

# 軌跡のグラフ
x_positions = [pos[0] for pos in positions]
y_positions = [pos[1] for pos in positions]

plt.subplot(2, 3, 4)
plt.plot(x_positions, label='X座標')
plt.plot(y_positions, label='Y座標')
plt.xlabel('フレーム番号')
plt.ylabel('位置')
plt.title('物体の軌跡')
plt.legend()
plt.grid(True)

plt.subplot(2, 3, 5)
plt.plot(x_positions)
plt.title('X方向の移動')
plt.xlabel('フレーム')
plt.ylabel('X座標')
plt.grid(True)

plt.subplot(2, 3, 6)
plt.plot(y_positions)
plt.title('Y方向の移動')
plt.xlabel('フレーム')
plt.ylabel('Y座標')
plt.grid(True)

plt.tight_layout()
plt.show()

## Challenge Problem: 連続的な動き検出

以下の要件を満たす動き検出システムを実装してください：

1. フレーム間差分法と背景差分法を組み合わせたハイブリッド手法
2. 動きが検出された場合、その領域を四角で囲む
3. 動きの速度に応じて色を変更（遅い=青、早い=赤）
4. 動きが一定時間以上続いている場合、アラートを出力

In [ ]:
class HybridMotionDetector:
    """ハイブリッド動き検出器"""
    def __init__(self, frame_diff_threshold=30, bg_diff_threshold=30, min_area=100):
        self.frame_diff_threshold = frame_diff_threshold
        self.bg_diff_threshold = bg_diff_threshold
        self.min_area = min_area
        
        # 背景差分器
        self.bg_subtractor = SimpleBackgroundSubtractor(
            alpha=0.95, threshold=bg_diff_threshold
        )
        
        # 動き履歴
        self.motion_history = []
        self.max_history = 30
    
    def detect(self, frame):
        """動きを検出し、結果を返す"""
        # 1. フレーム間差分
        if hasattr(self, 'prev_frame'):
            frame_diff = frame_difference(
                self.prev_frame, frame, self.frame_diff_threshold
            )
        else:
            frame_diff = np.zeros_like(frame)
        
        # 2. 背景差分
        self.bg_subtractor.update_background(frame)
        bg_diff = self.bg_subtractor.subtract(frame)
        
        # 3. 論理和（どちらかで検出された領域）
        combined = cv2.bitwise_or(frame_diff, bg_diff)
        
        # 4. 輪郭検出
        contours, _ = cv2.findContours(
            combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        
        # 5. 面積フィルタリング
        large_contours = [
            cnt for cnt in contours 
            if cv2.contourArea(cnt) > self.min_area
        ]
        
        # 動き領域を計算
        total_area = sum(cv2.contourArea(cnt) for cnt in large_contours)
        max_speed = min(255, total_area // 10)  # 速度の見積もり
        
        # 結果の画像を作成
        result = frame.copy()
        if len(frame.shape) == 2:
            result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
        
        # 動き領域を描画
        for cnt in large_contours:
            # 色を速度に応じて変更
            color_intensity = min(255, max_speed * 2)
            color = (color_intensity, 0, 255 - color_intensity)  # 青から赤へ
            
            # 輪郭を描画
            cv2.drawContours(result, [cnt], 0, color, 2)
            
            # バウンディングボックスを描画
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(result, (x, y), (x+w, y+h), color, 2)
        
        # 動き履歴を更新
        self.motion_history.append(total_area)
        if len(self.motion_history) > self.max_history:
            self.motion_history.pop(0)
        
        # アラートチェック（5フレーム以上継続）
        alert = False
        if len(self.motion_history) >= 5:
            if all(area > 0 for area in self.motion_history[-5:]):
                alert = True
                cv2.putText(result, "ALERT!", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # 前フレームを保存
        self.prev_frame = frame.copy()
        
        return result, {
            'total_area': total_area,
            'max_speed': max_speed,
            'alert': alert
        }

# テスト
detector = HybridMotionDetector()

plt.figure(figsize=(15, 10))

for idx, frame_num in enumerate([0, 5, 10, 15, 20, 25]):
    if frame_num < len(frames):
        result, info = detector.detect(frames[frame_num])
        
        plt.subplot(2, 3, idx + 1)
        plt.imshow(result, cmap='gray' if len(result.shape) == 2 else None)
        plt.title(f"フレーム {frame_num}\n面積: {info['total_area']}")
        plt.axis('off')

plt.tight_layout()
plt.show()

---

# Self-Check (理解度確認)

本日の学習内容を確認しましょう：

## 基礎知識
- [ ] 動き検出の基本的な概念と応用を理解した
- [ ] フレーム間差分法の原理と実装方法を理解した
- [ ] 背景差分法の原理と背景モデルの更新方法を理解した
- [ ] 光流法の基本的な考え方と動きベクトルを理解した

## 実践力
- [ ] フレーム間差分法を自前で実装できた
- [ ] 背景モデルを更新する複数の手法を比較できた
- [ ] 光流法を使った物体追跡を実装できた
- [ ] ハイブリッド動き検出システムを構築できた

## 深層理解
- [ ] 各手法の長所と短所を比較できた
- [ ] ノイズ除去の重要性を理解した
- [ ] 計算効率と精度のトレードオフを理解した
- [ ] 実際のアプリケーションへの応用方法を考えた

---

**お疲れ様でした！** Day 21はこれで終了です。

次回（Day 22）は「ニューラルネットワーク基礎」を学び、深層学習の基礎から始めます。

復習課題：
1. 自分のスマートフォンやウェブカメラからリアルタイムに動き検出を実装してみる
2. 異なる閾値値での動き検出性能を比較する
3. 背景の照明変化に対応するための改良方法を考える